In [ ]:
import os
from pathlib import Path
import glob
import numpy as np
import pandas as pd
import soundfile as sf
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy import signal

In [ ]:
def load_audio(file_path):
    """Load an audio file and return the sample rate and data."""
    sample_rate, data = wavfile.read(file_path)
    data = data.astype(np.float32)
    return sample_rate, data

In [ ]:
def ess_signal(f1, f2, T, sr, amp=0.95, fade=0.01):
    """
    Generate an Exponential Sine Sweep (ESS) signal.

    Parameters
    ----------
    f1 : float
        Start frequency of the sweep in Hz.
        Must be > 0.
        Determines the lowest frequency excited by the sweep.

    f2 : float
        End frequency of the sweep in Hz.
        Must satisfy f2 > f1 and f2 < sr / 2.
        Determines the highest frequency excited by the sweep.

    T : float
        Total duration of the sweep in seconds.
        Controls frequency resolution and SNR:
        longer T → more energy per frequency, better separation of nonlinearities.

    sr : int
        Sampling rate in Hz.
        Defines the discrete-time representation of x(t).
        Must match the playback and recording sample rate.

    amp : float, optional
        Linear amplitude scaling factor (0 < amp ≤ 1).
        Used to prevent digital clipping.
        amp = 1 corresponds to full-scale sine output.

    fade : float, optional
        Fraction of total duration used for fade-in and fade-out.
        Value is relative to T (e.g., 0.01 = 1% of samples).
        Prevents spectral leakage and audible clicks at sweep boundaries.

    Returns
    -------
    t : ndarray
        Time vector in seconds, shape (N,).
        t[n] = n / sr.

    x : ndarray
        ESS waveform, float32, shape (N,).
        Exponential frequency sweep from f1 to f2 over T seconds.
    """
    
    N = int(np.round(T * sr))
    t = np.arange(N) / sr

    K = T / np.log(f2 / f1)
    phase = 2 * np.pi * f1 * K * (np.exp(t / K) - 1.0)
    x = np.sin(phase)

    # Optional fade-in/out (raised-cosine)
    if fade and fade > 0:
        n_fade = int(np.round(fade * N))
        if n_fade > 1:
            w = np.ones(N)
            ramp = 0.5 - 0.5 * np.cos(np.pi * np.arange(n_fade) / n_fade)
            w[:n_fade] = ramp
            w[-n_fade:] = ramp[::-1]
            x *= w

    x = (amp * x).astype(np.float32)
    return t, x

def Hf(X, Y, eps=1e-12):
    """
    H(f) = Y(f) / X(f), with magnitude-scaled stabilization.
    X, Y are frequency-domain (complex) arrays of equal shape.
    """
    denom = X + eps * np.exp(1j * np.angle(X))
    return Y / denom

def Hf_reg(X, Y, eps=1e-12):
    """
    Regularized transfer function estimate:
    H = (Y * conj(X)) / (|X|^2 + eps)
    """
    return (Y * np.conj(X)) / (np.abs(X)**2 + eps)

def Mf(Hf, eps=1e-12):
    """
    Compute log-magnitude response in dB.

    Parameters
    ----------
    Hf : ndarray
        Complex transfer function H(f).
    eps : float, optional
        Floor to avoid log(0).

    Returns
    -------
    mag_db : ndarray
        Log-magnitude response in dB.
    """
    mag = np.abs(Hf)
    mag_db = 20.0 * np.log10(mag + eps)
    return mag_db